<a href="https://colab.research.google.com/github/chirazedrine/EmoGen/blob/main/EmoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Dowload dependencies

In [ ]:
!pip install torch torchvision

# Step 1: Download and extract the dataset

In [ ]:
!wget -O EmoSet-118K.zip "https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?dl=1&rlkey=7f3oyjkr6zyndf0gau7t140rv"


--2024-03-21 05:30:59--  https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?dl=1&rlkey=7f3oyjkr6zyndf0gau7t140rv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8bfe067a52773e8357b5b0bd79.dl.dropboxusercontent.com/cd/0/inline/CPcQ9gcZ3MSkym0Hy0Kv8FaQ7kQZTi7jVrIuQxQ5p2TkQz801w6fFXOKQF3nbVmkFA_Jkura9FnDupQ1AJWt_XlPrUUA-sf4LWwVKsaJfnHmb-K2KXKcTZtzhAi5bR2snE6n_SZrNKqVxFxd731lz9tt/file?dl=1# [following]
--2024-03-21 05:31:00--  https://uc8bfe067a52773e8357b5b0bd79.dl.dropboxusercontent.com/cd/0/inline/CPcQ9gcZ3MSkym0Hy0Kv8FaQ7kQZTi7jVrIuQxQ5p2TkQz801w6fFXOKQF3nbVmkFA_Jkura9FnDupQ1AJWt_XlPrUUA-sf4LWwVKsaJfnHmb-K2KXKcTZtzhAi5bR2snE6n_SZrNKqVxFxd731lz9tt/file?dl=1
Resolving uc8bfe067a52773e8357b5b0bd79.dl.dropboxusercontent.com (uc8bfe067a52773e8357b5b0bd79.dl.dropboxusercontent.com)... 16

In [ ]:
!unzip -q EmoSet-118K.zip -d /content/EmoSet-118K/

replace /content/EmoSet-118K/annotation/amusement/amusement_00000.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/EmoSet-118K/annotation/amusement/amusement_00001.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Step 2: Parse the JSON files to create a mapping for image paths and labels

In [ ]:
import os
import json
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn

base_path = '/content/EmoSet-118K'
annotations_dir = os.path.join(base_path, 'annotation')
images_dir = os.path.join(base_path, 'image')

# Parse JSON annotations
def parse_annotations(annotations_dir):
    annotations = []
    for emotion_dir_name in os.listdir(annotations_dir):
        emotion_dir_path = os.path.join(annotations_dir, emotion_dir_name)
        if os.path.isdir(emotion_dir_path):
            for annotation_file in os.listdir(emotion_dir_path):
                annotation_file_path = os.path.join(emotion_dir_path, annotation_file)
                with open(annotation_file_path, 'r') as f:
                    annotation_data = json.load(f)
                    annotations.append(annotation_data)
    return annotations

annotations = parse_annotations(annotations_dir)

# Verify the number of annotations loaded
print(f"Loaded {len(annotations)} annotations.")

# Extract unique emotions and create a mapping to indices
unique_emotions = sorted({anno['emotion'] for anno in annotations})
emotion_to_idx = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
print(f"Unique emotions found: {list(emotion_to_idx.keys())}")

class EmoSetDataset(Dataset):
    def __init__(self, annotations, images_dir, transform=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        image_path = os.path.join(self.images_dir, annotation['emotion'], f"{annotation['image_id']}.jpg")
        image = Image.open(image_path).convert('RGB')
        label = emotion_to_idx[annotation['emotion']]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset and DataLoader
dataset = EmoSetDataset(annotations, images_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


NameError: name '_C' is not defined

# Step 3-4: Define the CNN model for emotion classification

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn, optim

# Load pre-trained EfficientNet-B0 model
weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights)

# Replace the classifier to match the number of unique emotions
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(unique_emotions))

# Setup device: Use GPU if available, else fall back to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Move model to the appropriate device
model = model.to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Step 5: Train the model -- deprecated

In [ ]:
!pip install tqdm

##Adjust training loop for early stopping

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from tqdm import tqdm

# Assuming annotations is a list of dictionaries, split it into training and validation
train_annos, val_annos = train_test_split(annotations, test_size=0.2, random_state=42)

# Create datasets for training and validation
train_dataset = EmoSetDataset(train_annos, images_dir, transform=train_transforms)
val_dataset = EmoSetDataset(val_annos, images_dir, transform=transform)  # Use the original transform for validation

# Data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define early stopping parameters
early_stopping_patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0

    for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Validation'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    # Early stopping logic
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
        # Save the best model
        torch.save(model.state_dict(), '/content/drive/My Drive/EmoGen/best_model.pth')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print('Early stopping triggered.')
            break

## Step 5.5: Test data

In [ ]:
import os
import shutil
from random import sample

# Base directory for dataset images
base_path = '/content/EmoSet-118K/image'
# Directory for test images
test_dir = '/content/test'

# Create test directory if it doesn't exist
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    print(f"Created test directory: {test_dir}")
else:
    print(f"Test directory already exists: {test_dir}")

# Get all images from each emotion subdirectory
all_images = []
emotion_subdirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

for emotion_subdir in emotion_subdirs:
    emotion_path = os.path.join(base_path, emotion_subdir)
    images = [os.path.join(emotion_path, img) for img in os.listdir(emotion_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    all_images.extend(images)

# Sample 100 images or the total number of images if less than 100
num_images_to_select = min(100, len(all_images))
selected_images = sample(all_images, num_images_to_select)

# Copy selected images to the test directory, maintaining the emotion subdirectory structure
for img_path in selected_images:
    emotion_subdir = os.path.basename(os.path.dirname(img_path))
    destination_subdir = os.path.join(test_dir, emotion_subdir)
    if not os.path.exists(destination_subdir):
        os.makedirs(destination_subdir)
    shutil.copy(img_path, destination_subdir)

print(f"Completed preparing the test set with {num_images_to_select} images in: {test_dir}")


# Step 6: Evaluate the model -- deprecated

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Assuming the EmoSetDataset class and emotion_to_idx have been defined in earlier steps
# Define the transformation applied to the test images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Prepare the test dataset
test_annotations = [{
    'emotion': os.path.basename(os.path.dirname(img_path)),
    'image_id': os.path.splitext(os.path.basename(img_path))[0]
} for img_path in selected_images]

# Initialize the test dataset with the test_dir as the images_dir
test_dataset = EmoSetDataset(test_annotations, images_dir=test_dir, transform=transform)

# Initialize the DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

running_corrects = 0
total_samples = 0

for inputs, labels in tqdm(test_loader, desc="Evaluating"):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels).item()
    total_samples += labels.size(0)

test_acc = running_corrects / total_samples
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
print(f"Number of test annotations: {len(test_annotations)}")
print("Sample test annotations:", test_annotations[:3])

# Step 5-6: training and validation


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import json
from PIL import Image

# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item() / labels.size(0)

# Load pre-trained EfficientNet-B0 model
weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights)

# Replace the classifier to match the number of unique emotions
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(unique_emotions))

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data augmentation and normalization for training
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Transformations for validation and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Split annotations into training and validation
train_annos, val_annos = train_test_split(annotations, test_size=0.2, random_state=42)

# Create datasets and data loaders for training and validation
train_dataset = EmoSetDataset(train_annos, images_dir, transform=train_transforms)
val_dataset = EmoSetDataset(val_annos, images_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Training and validation loop with early stopping
num_epochs = 5
best_val_accuracy = 0

for epoch in range(num_epochs):
    model.train()
    total_train_loss, total_train_accuracy = 0, 0

    for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * inputs.size(0)
        total_train_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)

    epoch_loss = total_train_loss / len(train_loader.dataset)
    epoch_accuracy = total_train_accuracy / len(train_loader.dataset)
    print(f'Training - Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    # Validation phase
    model.eval()
    total_val_loss, total_val_accuracy = 0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Validation'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item() * inputs.size(0)
            total_val_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)

    val_epoch_loss = total_val_loss / len(val_loader.dataset)
    val_epoch_accuracy = total_val_accuracy / len(val_loader.dataset)
    print(f'Validation - Epoch {epoch+1}, Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.4f}')

    if val_epoch_accuracy > best_val_accuracy:
        best_val_accuracy = val_epoch_accuracy
        torch.save(model.state_dict(), 'best_model.pth')
        print("Best model saved with Accuracy:", best_val_accuracy)

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
test_running_corrects = 0

for inputs, labels in tqdm(test_loader, desc="Evaluating"):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
        test_running_corrects += calculate_accuracy(outputs, labels) * inputs.size(0)

test_accuracy = test_running_corrects / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Step 7: Predict emotion for new images

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

model_path = '/content/drive/My Drive/EmoGen/best_model.pth'
model.load_state_dict(torch.load(model_path))
model.eval()

def predict_emotion(image_path, model):
     idx_to_emotion = {
        0: 'Amusement',
        1: 'Anger',
        2: 'Awe',
        3: 'Contentment',
        4: 'Disgust',
        5: 'Excitement',
        6: 'Fear',
        7: 'Sadness'
    }
    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted_idx = torch.max(outputs, 1)
        predicted_emotion = idx_to_emotion[predicted_idx.item()]
    return predicted_emotion

image_path = '/content/Inputs/image.jpg'
predicted_emotion = predict_emotion(image_path, model)
print(f"Predicted emotion: {predicted_emotion}")

# Step 8: Generate plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the data for epochs, training/validation loss, and training/validation accuracy
epochs = np.arange(1, 6)
train_loss = [1.2100, 1.0865, 1.0342, 0.9934, 0.9656]
val_loss = [1.0062, 0.9603, 0.9240, 0.8709, 0.8657]
train_accuracy = [56.41, 60.60, 62.53, 64.06, 65.03]
val_accuracy = [64.57, 65.77, 67.43, 68.31, 68.89]

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, 'o-', label='Training Loss')
plt.plot(epochs, val_loss, 'o-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.savefig("/content/loss_plot.png")

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_accuracy, 'o-', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'o-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.savefig("/content/accuracy_plot.png")

# Show the plots
plt.show()

# Return paths to the generated plots
loss_plot_path = "/mnt/data/loss_plot.png"
accuracy_plot_path = "/mnt/data/accuracy_plot.png"
(loss_plot_path, accuracy_plot_path)